# Feature Extraction for EMG signals

## using LSTM

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from models import LSTM_Emb_Classifier, EMG_Feature_Extractor
from utils.loaders import ActionNetEmgDataset
from torch.utils.data import DataLoader
from utils.logger import logger
from tqdm import tqdm

/Users/andreavannozzi/GithubProjects/Multimodal-Egocentric-Action-Recognition/env/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: dlopen(/Users/andreavannozzi/GithubProjects/Multimodal-Egocentric-Action-Recognition/env/lib/python3.8/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: <0EB69795-4559-3C98-9EA1-35B6A988BB99> /Users/andreavannozzi/GithubProjects/Multimodal-Egocentric-Action-Recognition/env/lib/python3.8/site-packages/torchvision/image.so
  Expected in:     <E4E2FFCA-031E-3974-A7B0-45408D7F4956> /Users/andreavannozzi/GithubProjects/Multimodal-Egocentric-Action-Recognition/env/lib/python3.8/site-packages/torch/lib/libtorch_cpu.dylib
  warn(f"Failed to load image Python extension: {e}")


### Training

In [2]:
BATCH_SIZE = 32
LR = 0.0001
MOMENTUM = 0.9
WEIGHT_DECAY = 1e-4
STEP_SIZE = 10
GAMMA = 0.1
NUM_EPOCHS = 50

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
if torch.backends.mps.is_available():
    DEVICE = 'mps'
    logger.info("------ USING APPLE SILICON GPU ------")
 

2024-06-07 11:34:06 LOG INFO ------ USING APPLE SILICON GPU ------


In [7]:
# Parametri del modello
input_dim = 16*25
hidden_dim = 256
embedding_dim = 128
output_dim = 20  # Definisci il numero di classi

# Inizializzazione del modello, della loss function e dell'ottimizzatore
model = LSTM_Emb_Classifier(input_dim=input_dim, hidden_dim=hidden_dim, embedding_dim=embedding_dim, num_class=output_dim)
model = model.to(DEVICE)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=STEP_SIZE, gamma=GAMMA)

train_dataset = ActionNetEmgDataset('train', 25, 5, True, './action-net', "./action-net/saved_emg", 2) # Inserisci il path al dataset di training
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True, drop_last=True) # Inserisci il dataloader per il training

val_dataset = ActionNetEmgDataset('test', 25, 5, True, './action-net', "./action-net/saved_emg", 2)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True, drop_last=True)

logger.info(f"Model: {model}")
logger.info(f"len train_dataset: {len(train_dataset)}")
logger.info(f"len train_loader: {len(train_loader)}")


/Users/andreavannozzi/GithubProjects/Multimodal-Egocentric-Action-Recognition/env/lib/python3.8/site-packages/torch/nn/modules/rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
2024-06-07 11:40:24 LOG INFO Model: LSTM_Emb_Classifier(
  (lstm): LSTM(400, 256, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=256, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=20, bias=True)
)
2024-06-07 11:40:24 LOG INFO len train_dataset: 1795
2024-06-07 11:40:24 LOG INFO len train_loader: 56
2024-06-07 11:40:24 LOG INFO len val_dataset: <torch.utils.data.dataloader.DataLoader object at 0x32231ddf0>


In [5]:
def evaluate(model, data_loader, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for x, y in data_loader:
            x = x.reshape(BATCH_SIZE, 5, 25, -1)
            x = x.permute(1, 0, 2, 3)
            y = y.to(device)
            
            for i in range(5):
                x_t = x[i].to(device)
                outputs, embeddings = model(x_t)
                _, predicted = torch.max(outputs, 1)
                total += y.size(0)
                correct += (predicted == y).sum().item()

    accuracy = correct / total
    return accuracy


In [6]:
model.train()
for epoch in range(NUM_EPOCHS):
        model.train()
        epoch_loss = [0.0, 0]
        for i_val,(x, y) in tqdm(enumerate(train_loader)):
            x = x.reshape(BATCH_SIZE, 5, 25, -1)
            x = x.permute(1, 0, 2, 3)
            y = y.to(DEVICE)
            #logger.info(f"X: {x[0][0]}")
            # Category Loss
            #logger.info(f"X: {x.size()}")
            for i in range(5):
                x_t = x[i].to(DEVICE)
                outputs, embeddings = model(x_t)
                # Log details about the outputs
                #logger.info(f"Output type: {cls_o.logits.shape}")
                
                criterion = nn.CrossEntropyLoss()
                loss = criterion(outputs, y.long())

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                epoch_loss[0] += loss.item()
                epoch_loss[1] += x.size(0)

                if (i_val + 1) % (len(train_loader) // 5) == 0:
                    logger.info("[{}/{}]".format(i_val + 1, len(train_loader)))
            
        scheduler.step()
        logger.info(f'[EPOCH {epoch+1}] Avg. Loss: {epoch_loss[0] / epoch_loss[1]}')


        #save checkpoint in a file
        if (epoch+1) % 10 == 0:
            train_accuracy = evaluate(model, train_loader, DEVICE)
            val_accuracy = evaluate(model, val_loader, DEVICE)
            logger.info(f'[EPOCH {epoch+1}] Train Accuracy: {train_accuracy}')
            logger.info(f'[EPOCH {epoch+1}] Val Accuracy: {val_accuracy}')
            torch.save(model.state_dict(), f'./saved_models/LSTM_Emb_Classifier/final_LSTM_Emb_epoch_{epoch+1}.pth')
        if (epoch+1) % STEP_SIZE == 0:
            logger.info(f'Current LR: {scheduler.get_last_lr()}')

TypeError: 'module' object is not callable

### Extraction

In [ ]:
# ----------TO ADJUST-------------
extraction_dataset = ""
extraction_loader = ""


# Estrazione degli embeddings per i dati completi
model.load_state_dict(torch.load(f'./saved_models/LSTM_Emb_Classifier/final_LSTM_Emb_epoch_50.pth'))
with torch.no_grad():
    all_embeddings = []
    for i_val,(x, y) in tqdm(enumerate(extraction_loader)):
        _, embeddings = model(x)
        all_embeddings.append(embeddings)
    all_embeddings = torch.cat(all_embeddings)

print(all_embeddings.shape)  # Controlla la forma degli embeddings

## using Not Neural Network Feature Extraction